In [2]:
!pip install pandas google-cloud-bigquery
!pip install pandas-gbq
import pandas as pd
from google.oauth2 import service_account
import pandas as pd
import pandas_gbq as gbq
credentials_path = '../token.json'

# Authenticate with your credentials
credentials = service_account.Credentials.from_service_account_file(
    credentials_path, scopes=['https://www.googleapis.com/auth/bigquery'])

# Set the credentials for pandas_gbq
gbq.context.credentials = credentials

In [155]:
from pandas_gbq import read_gbq
merged_data = read_gbq(f"""
SELECT *
FROM `capstone-398012.capstone.filled_table`

""",project_id='capstone-398012', dialect='standard')

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [165]:
#list of columns that contain NA 
na_cols = ['liqnonfinlevel', 'liqnonfintrend', 'dtdlevel', 'dtdtrend', 'DTDmedianNonFin', 'Sector_Number', 'DTDmedianFin', 'sigma', 'StkIndx']

new_df = merged_data.copy()
new_df['Event_type'] = new_df['Event_type'].map(lambda x : 0 if x ==2  else x)
#create new columns (_notNA) for each column that contains na values where 1 if original column is not na and 0 otherwise
for col in na_cols:
    new_col = col + '_notNA'
    new_df[new_col] = new_df[col].notna().astype(int)

grouped_df = new_df.groupby('CompNo')
grouped_df[na_cols].mean()
mean_values = grouped_df[na_cols].transform('mean')

new_df[na_cols] = new_df[na_cols].fillna(mean_values)
print(new_df['Event_type'].sum())
new_df.isna().sum().sort_values(ascending=False)
print(new_df['Event_type'].sum())



75
75


In [166]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
numerical = ['StkIndx', 'STInt', 'dtdlevel', 'dtdtrend', 'liqnonfinlevel', 'liqnonfintrend', 'ni2talevel', 'ni2tatrend', 'sizelevel', 'sizetrend', 'm2b', 'sigma', 'liqfinlevel', 'lqfintrend', 'DTDmedianFin', 'DTDmedianNonFin', 'dummy297fin']
categorical= [
 'Sector_Number',
 'Exchange',
 'liqnonfinlevel_notNA',
 'liqnonfintrend_notNA',
 'dtdlevel_notNA',
 'dtdtrend_notNA',
 'DTDmedianNonFin_notNA',
 'Sector_Number_notNA',
 'DTDmedianFin_notNA',
 'sigma_notNA',
 'StkIndx_notNA']
y = new_df['Event_type']
X = new_df.drop(['Event_type', 'CompNo', 'yyyy', 'mm'], axis=1)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train[numerical]), columns=numerical)
X_train_scaled.index = X_train.index
X_train_scaled = pd.concat([X_train_scaled, X_train[categorical]] ,axis=1)
X_test_scaled = pd.DataFrame(scaler.transform(X_test[numerical]), columns=numerical)
X_test_scaled.index = X_test.index
X_test_scaled = pd.concat([X_test_scaled, X_test[categorical]], axis=1)


## Logistic Regression

In [168]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix, classification_report
class_weights = {0:1, 1:10}
model = LogisticRegression(class_weight = class_weights)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R-squared:", r2)
coefficients = model.coef_
intercept = model.intercept_


conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)
classification_rep = classification_report(y_test, y_pred)
print("\nClassification Report:")
print(classification_rep)

Mean Squared Error: 0.02247191011235955
R-squared: -1.5226757369614514
Confusion Matrix:
[[1297   26]
 [   4    8]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1323
           1       0.24      0.67      0.35        12

    accuracy                           0.98      1335
   macro avg       0.62      0.82      0.67      1335
weighted avg       0.99      0.98      0.98      1335



/Users/thomastan/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Linear Regression


In [161]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

model = LinearRegression()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R-squared:", r2)
coefficients = model.coef_
intercept = model.intercept_
print("Coefficients:", coefficients)
print("Intercept:", intercept)

Mean Squared Error: 0.007632149265912448
R-squared: 0.14322201905764587
Coefficients: [-0.00375393 -0.06441143 -0.00042193  0.00141503 -0.00767383  0.0026687
  0.00420604  0.00046814 -0.01633283 -0.00045575  0.00038404 -0.00825186
  0.00168323 -0.00028511 -0.04249975  0.0243767   0.05637428 -0.00145505
 -0.11732824  0.          0.          0.          0.          0.
  0.          0.          0.          0.        ]
Intercept: 1.8148447662827003
